In [ ]:
import os
import cv2
import time
from collections import Counter
from ultralytics import YOLO
import easyocr
import re  # Import regex module

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Define paths
input_path = r'C:/Users/Abad/Desktop/FYP/yolo_test/combine/input_output/input/2.mp4'
output_dir = r"C:/Users/Abad/Desktop/FYP/yolo_test/combine/input_output/output\predict"
trained_model_path = "best.pt"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load YOLO model
try:
    model = YOLO(trained_model_path)
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Define province names to filter out
provinces = {"PUNJAB", "SINDH", "KPK", "BALOCHISTAN", "ISLAMABAD"}

# Predefined license plate formats (regex patterns)
# Single-line formats: "ABC-123" or "ABC 123"
# Double-line formats: "ABC-12 1234", "ABC 1234", or "ABC 12 1234"
license_plate_formats = [
    r"^[A-Z]{3}[- ]?\d{3}$",  # Single-line format (ABC-123 or ABC 123)
    r"^[A-Z]{3}[- ]?\d{2} \d{4}$",  # Double-line format (ABC-12 1234)
    r"^[A-Z]{3} \d{4}$",  # Double-line format (ABC 1234)
    r"^[A-Z]{3} \d{2} \d{4}$",  # Double-line format (ABC 12 1234)
]

# Store detected license plates
license_plate_counts = Counter()

def refine_license_text(text):
    """
    Refines the detected license plate text by:
    1. Removing province names.
    2. Cleaning unwanted characters.
    3. Matching against predefined formats.
    Args:
        text (str): The raw OCR-detected text.
    Returns:
        str: The refined license plate text.
    """
    # Remove province names
    for province in provinces:
        text = text.replace(province, "")
    
    # Remove unwanted characters and spaces
    cleaned_text = re.sub(r"[^A-Z0-9-]", "", text.upper())
    
    # Try to match the cleaned text with predefined formats
    for pattern in license_plate_formats:
        match = re.match(pattern, cleaned_text)
        if match:
            return match.group(0)  # Return the matched text
    
    # If no format matches, return the cleaned text as-is
    return cleaned_text

def process_frame(frame):
    results = model.predict(frame, conf=0.5)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            license_plate = frame[y1:y2, x1:x2]

            try:
                ocr_results = reader.readtext(license_plate)
                extracted_texts = [res[1].strip().upper() for res in ocr_results]  # Convert to uppercase
                
                # Join extracted texts and refine using predefined formats
                raw_text = " ".join(extracted_texts)
                license_text = refine_license_text(raw_text)
                
                if license_text:
                    license_plate_counts[license_text] += 1
            except Exception as e:
                print(f"OCR Error: {e}")
                license_text = "OCR Error"

            # Draw bounding box and text on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, license_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            print(f"Detected License Plate: {license_text}")
    
    return frame


if input_path.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".gif")):
    frame = cv2.imread(input_path)
    if frame is None:
        print(f"Error: Could not open image {input_path}")
        exit()
    
    processed_frame = process_frame(frame)
    output_image_path = os.path.join(output_dir, f"output_{os.path.basename(input_path)}")
    cv2.imwrite(output_image_path, processed_frame)
    print(f"Output image saved at: {output_image_path}")

    cv2.imshow("License Plate Detection", processed_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {input_path}")
        exit()
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    output_video_path = os.path.join(output_dir, f"output_{timestamp}.mp4")

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    if not out.isOpened():
        print(f"Error: Could not create output video file {output_video_path}")
        cap.release()
        exit()

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    current_frame = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        current_frame += 1
        print(f"Processing frame {current_frame}/{total_frames}...")

        processed_frame = process_frame(frame)
        out.write(processed_frame)

        cv2.imshow("License Plate Detection", processed_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Inference completed. Video saved at: {output_video_path}")

# Determine the most frequently detected license plate
if license_plate_counts:
    most_common_plate = license_plate_counts.most_common(1)[0]
    print(f"Most frequent license plate detected: {most_common_plate[0]} ({most_common_plate[1]} times)")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Processing frame 1/368...

0: 384x640 (no detections), 125.4ms
Speed: 0.0ms preprocess, 125.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2/368...

0: 384x640 (no detections), 93.7ms
Speed: 2.8ms preprocess, 93.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 3/368...

0: 384x640 (no detections), 99.1ms
Speed: 0.0ms preprocess, 99.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 4/368...

0: 384x640 (no detections), 93.4ms
Speed: 3.2ms preprocess, 93.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 5/368...

0: 384x640 (no detections), 83.5ms
Speed: 0.0ms preprocess, 83.5ms inference, 10.3ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 6/368...

0: 384x640 (no detections), 74.7ms
Speed: 8.9ms preprocess, 74.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 7/368...

0: 384x640 (no d